In [1]:
%matplotlib inline

import os
import sys

import gym
import matplotlib
import numpy as np

from collections import defaultdict

from racetrack_env import RacetrackEnv, Map

matplotlib.style.use('ggplot')

N_EPISODE = 1000000
MAX_STEP = 70
SAVE_FILE = 'racetrack-offpolicy.sav'

In [2]:
!rm $SAVE_FILE

In [3]:
with open('racetrack_map_4.txt', 'r') as f:
    amap = Map(f.read(), v_mgn=2, h_mgn=2)
vel_info = (
    0, 2,  # vx min/max
    -2, 2   # vy min/max
)
env = RacetrackEnv(amap, vel_info, MAX_STEP)

In [4]:
def create_random_policy(env):
    """
    Creates a random policy function.
    
    Args:
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities
    """
    nA = env.action_space.n
    A = np.ones(nA, dtype=float) / nA

    def policy_fn(observation):
        return A
    
    return policy_fn

In [5]:
def create_greedy_policy(Q):
    """
    Creates a greedy policy based on Q values.
    
    Args:
        Q: A dictionary that maps from state -> action values
        
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities.
    """
    
    def policy_fn(state):
        A = np.zeros_like(Q[state], dtype=float)
        best_action = np.argmax(Q[state])
        A[best_action] = 1.0
        return A
    return policy_fn

In [6]:
def mc_control_importance_sampling(env, num_episodes, behavior_policy, discount_factor=1.0):
    """
    가중 중요 샘플링을 이용한 몬테카를로 Off 정책 컨트롤. greedy한 최적 정책을 찾는다.
    Monte Carlo Control Off-Policy Control using Weighted Importance Sampling.
    Finds an optimal greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Nubmer of episodes to sample.
        behavior_policy: The behavior to follow while generating episodes.
            A function that given an observation returns a vector of probabilities for each action.
        discount_factor: Lambda discount factor.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities. This is the optimal greedy policy.
    """
    
    # The final action-value function.
    # A dictionary that maps state -> action values
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # The cumulative denominator of the weighted importance sampling formula
    # (across all episodes)
    C = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # Our greedily policy we want to learn
    target_policy = create_greedy_policy(Q)
        
    for i_episode in range(1, num_episodes + 1):
        # Print out which episode we're on, useful for debugging.
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()

        # Generate an episode.
        # An episode is an array of (state, action, reward) tuples
        episode = []
        state = env.reset()
        for t in range(MAX_STEP):
            # print(t)
            # Sample an action from our policy
            probs = behavior_policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
            #if np.random.randint(10000) == 0:
            #print("** state {}, probs {}, action {}, next_state {}, reward {}".format(state, probs, action, next_state, reward))            
            # print("state {}, probs {}, action {}, next_state {}, done {}, reward {}".format(state, probs, action, next_state, done, reward))            
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
        
        # Sum of discounted returns
        G = 0.0
        # The importance sampling ratio (the weights of the returns)
        W = 1.0
        # For each step in the episode, backwards
        # print("Start")
        for t in range(len(episode))[::-1]:
            state, action, reward = episode[t]
            # Update the total reward since step t
            G = discount_factor * G + reward
            # Update weighted importance sampling formula denominator
            C[state][action] += W
            # Update the action-value function using the incremental update formula (5.7)
            # This also improves our target policy which holds a reference to Q
            # Q가 업데이트되면 타겟 정책도 향상
            qd = (W / C[state][action]) * (G - Q[state][action])
            #if np.random.randint(10000) == 0:
            #print("1  state {}, G {}, W {}, C {}, Q {}, qd {}, action {}, reward {}".format(state, G, W, C[state][action], Q[state][action], qd, action, reward))
            Q[state][action] += qd
            # print("  state {}, G {}, W {}, C {}, Q {}, action {}, reward {}".format(state, G, W, C[state][action], Q[state][action], action, reward))
            # If the action taken by the behavior policy is not the action 
            # taken by the target policy the probability will be 0 and we can break
            # 행위 정책에 의한 현재 동작이 타겟 정책에서는 나오지 않으면 멈추고 새로운 에피소드로
            if action !=  np.argmax(target_policy(state)):
                # print("  behavior policy {} not equal to target policy in {}. skip".format(action, state))
                break
            bepol = behavior_policy(state)[action]
            W = W * 1./bepol
        
    return Q, target_policy

In [7]:
if os.path.isfile(SAVE_FILE):
    Q, policy = env.load(SAVE_FILE, create_greedy_policy)
else:
    random_policy = create_random_policy(env)
    Q, policy = mc_control_importance_sampling(env, num_episodes=N_EPISODE,
                                               behavior_policy=random_policy)
    env.save(Q, SAVE_FILE)
    policy = create_greedy_policy(Q)    

Episode 1000000/1000000.

In [ ]:
print(env.score(policy))

0.0


In [ ]:
env.play(policy)

turn 12/70, state (1, 8, 0, 0), action, 0, reward -1, done False
#######################################################################
#######################################################################
#################################################################ffff##
###################################........####################ffffff##
#################################.............################..fffff##
################################...............##############....fff###
###############################.................############......#####
#############################.......#####.......############.....######
#############################.....#########.....############....#######
##O......###################.....###########....###########.....#######
##s........################.....############....###########....########
##s.........##############.....#############....###########....########
########.....###########......###############...###########....########

In [ ]:
len(Q)